In [8]:
import pandas as pd

In [9]:
df_gastro = pd.read_csv("merged_df_UmsatzTemperatur.csv")
df_dax = pd.read_csv("DAXHISTORIE.csv")

#Umwandlung des Jahr_Monat von 2023-1 zu 2023-01
df_gastro['Jahr_Monat'] = df_gastro['Jahr_Monat'].apply(lambda x: f"{x.split('-')[0]}-{int(x.split('-')[1]):02d}")


In [23]:
df_gastro

,Unnamed: 0,ID,Jahr,Monat,Gastro_Id,Umsatz in millionen,TMK,Jahr_Monat
0,0,0,1994,1,WZ08-55,92.3,4.2,1994-01
1,1,1,1994,1,WZ08-551,91.9,4.2,1994-01
2,2,2,1994,1,WZ08-552,97.3,4.2,1994-01
3,3,3,1994,1,WZ08-553,68.4,4.2,1994-01
4,4,4,1994,1,WZ08-559,127.1,4.2,1994-01
...,...,...,...,...,...,...,...,...
3724,3724,3955,2023,12,WZ08-561,94.3,6.5,2023-12
3725,3725,3956,2023,12,WZ08-562,96.5,6.5,2023-12
3726,3726,3957,2023,12,WZ08-563,70.1,6.5,2023-12
3727,3727,3958,2023,12,WZ08-561-01,91.9,6.5,2023-12


In [11]:
df_dax

,Date,Open,High,Low,Close,Adj Close,Volume
0,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0
1,1987-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,1988-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0
4,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0
...,...,...,...,...,...,...,...
9372,2024-05-28,18775.550781,18855.050781,18635.039063,18677.869141,18677.869141,52670400.0
9373,2024-05-29,18677.699219,18677.699219,18438.009766,18473.289063,18473.289063,59553600.0
9374,2024-05-30,18465.470703,18531.849609,18394.429688,18496.789063,18496.789063,52348300.0
9375,2024-05-31,18495.220703,18542.269531,18426.720703,18497.939453,18497.939453,144013800.0


In [12]:
df_dax['Date'] = pd.to_datetime(df_dax['Date'])

# Jahr und Monat extrahieren und als neue Spalte hinzufügen
df_dax['YearMonth'] = df_dax['Date'].dt.to_period('M')

# Gruppieren nach Jahr und Monat und aggregieren
aggregated_dax = df_dax.groupby('YearMonth').agg({
    'Open': 'first',
    'Close': 'last',
}).reset_index()

# Umbenennen der 'YearMonth' Spalte in 'Date' für Konsistenz
aggregated_dax['YearMonth'] = aggregated_dax['YearMonth'].astype(str)  # Umwandlung in String-Format 'YYYY-MM'
aggregated_dax.rename(columns={'YearMonth': 'Date'}, inplace=True)
aggregated_dax

,Date,Open,Close
0,1987-12,1005.190002,1005.190002
1,1988-01,956.489990,936.000000
2,1988-02,951.500000,1083.339966
3,1988-03,1087.319946,1062.329956
4,1988-04,1063.280029,1049.170044
...,...,...,...
434,2024-02,16834.130859,17678.189453
435,2024-03,17793.470703,18492.490234
436,2024-04,18492.359375,17932.169922
437,2024-05,17935.779297,18497.939453


In [16]:
min = df_gastro["Jahr_Monat"].min()
min

'1994-1'

In [17]:
max = df_gastro["Jahr_Monat"].max()
max

'2023-9'

In [20]:
# Step 1: Determine the date range from df_gastro
min_date = df_gastro["Jahr_Monat"].min()
max_date = df_gastro["Jahr_Monat"].max()

# Step 2: Filter the aggregated_dax DataFrame
filtered_dax = aggregated_dax[(aggregated_dax['Date'] >= min_date) & (aggregated_dax['Date'] <= max_date)]

# Step 3: Drop rows with any missing values
filtered_dax = filtered_dax.dropna()

# Display the filtered DataFrame
filtered_dax


,Date,Open,Close
82,1994-10,1995.800049,2061.580078
83,1994-11,2052.219971,2057.280029
84,1994-12,2054.159912,2097.510010
85,1995-01,2091.469971,2024.819946
86,1995-02,2028.699951,2097.850098
...,...,...,...
428,2023-08,16414.789063,15947.080078
429,2023-09,15936.339844,15386.580078
430,2023-10,15439.959961,14810.339844
431,2023-11,14851.959961,16215.429688


#### merge von aggregated_dax und gastro

In [22]:
merged_df_dax_gastro = pd.merge(df_gastro, filtered_dax, how='left', left_on='Jahr_Monat', right_on='Date')
merged_df_dax_gastro

,Unnamed: 0,ID,Jahr,Monat,Gastro_Id,Umsatz in millionen,TMK,Jahr_Monat,Date,Open,Close
0,0,0,1994,1,WZ08-55,92.3,4.2,1994-01,NaN,NaN,NaN
1,1,1,1994,1,WZ08-551,91.9,4.2,1994-01,NaN,NaN,NaN
2,2,2,1994,1,WZ08-552,97.3,4.2,1994-01,NaN,NaN,NaN
3,3,3,1994,1,WZ08-553,68.4,4.2,1994-01,NaN,NaN,NaN
4,4,4,1994,1,WZ08-559,127.1,4.2,1994-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3724,3724,3955,2023,12,WZ08-561,94.3,6.5,2023-12,2023-12,16296.919922,16751.640625
3725,3725,3956,2023,12,WZ08-562,96.5,6.5,2023-12,2023-12,16296.919922,16751.640625
3726,3726,3957,2023,12,WZ08-563,70.1,6.5,2023-12,2023-12,16296.919922,16751.640625
3727,3727,3958,2023,12,WZ08-561-01,91.9,6.5,2023-12,2023-12,16296.919922,16751.640625


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming merged_df_dax_gastro is already created and contains the necessary columns

# Filter out rows with missing values in the relevant columns
merged_df_dax_gastro = merged_df_dax_gastro.dropna(subset=['Umsatz in millionen', ' TMK', 'Open', 'Close'])

# Define features and target variable
features = merged_df_dax_gastro[[' TMK', 'Open', 'Close']]
target = merged_df_dax_gastro['Umsatz in millionen']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict using the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# Display the coefficients of the model
coefficients = pd.DataFrame(model.coef_, features.columns, columns=['Coefficient'])
print(coefficients)


Mean Squared Error: 1001.4031983648636
R^2 Score: 0.2877456580184321
       Coefficient
 TMK      1.874705
Open      0.001400
Close    -0.005539
